## Exploration of Gradient Descent and Neural Networks

The purpose of this notebook is to translate the mathematical ideas in
gradient descent into code.

We start with a 1-wide neural network and build up from there.

### Notation

The following conventions will be used:

| Variable or symbol | Definition |
|---|---|
| *w* | A single weights matrix that feeds into a hidden layer.  `w` is an `(m, n)` matrix, where *n* is the width (size) of the inputs, and *m* is the size of the outputs (the width of the hidden layer). |
| `w1` | When using a fixed number of layers *L*, `w1` will be the weights feeding into the first hidden layer. |
| *b* | A single bias matrix that contributes to a hidden layer.  `b` is a `(m, 1)` matrix, where *m* is the size (width) of the associated hidden layer.  |
| `b1` | When using a fixed number of layers *L1*, `b1` will be the weights feeding into the first hidden layer. |
| `z` | A preactivated layer.  It is the result of the linear combination (weights times inputs plus bias) before an activation function is applied. |
| `a` | An activated layer, i.e. `f(z)`. |
| *L* | The depth of the neural network; the number of hidden layers plus the output layer. |


Note that when using a fixed number of layers *L > 1*, the *input layer* is treated as the “0th” layer and does not have wieghts or biases.
Think of this like each layer owning the weights and biases that feed into it.

### TODO: !Important!  Add a section describing how NN depth + width and training data size are reflected in the dimensionality of my datastructures

What is `w[0]`, `w[0][0]`, `b[0]`, `z[0]`?

### 1-Wide Neural Network (L = 1)
#### Two layered network

Start with the simplest NN - a single input node connected to a single output node.


In [10]:
import numpy as np

def init_params():
    w1 = np.random.randn(1) * 0.01
    b1 = np.random.randn(1)
    return w1, b1

init_params()

array([-0.11042608])